# Checklist de Conformidade
Este notebook demonstra, ponto a ponto, como o código atende aos requisitos do projeto.

## 1. Framework – TensorFlow 2 / Keras

In [ ]:
import tensorflow as tf
print('TensorFlow version:', tf.__version__)

## 2. Divisão Temporal 70 / 15 / 15 – sem embaralhar
A função `train_val_test_split` mantém a ordem dos timestamps; o embaralhamento ocorre **apenas** dentro do `tf.data` para o treino. O trecho abaixo carrega um *sample* e mostra o tamanho das partições.

In [ ]:
from pathlib import Path
import numpy as np
from codes.data_utils import load_csv_parts, window_generator, train_val_test_split

# Pequeno subset para demonstração rápida
df = load_csv_parts(Path('dataset') )[:1000]  # carrega primeiras 1000 linhas
X, y = window_generator(df, window_size=240, step_size=10)
X_train, y_train, X_val, y_val, X_test, y_test = train_val_test_split(X, y)
print('Train:', X_train.shape[0], 'Val:', X_val.shape[0], 'Test:', X_test.shape[0])

## 3. Janela (WINDOW) e STRIDE
- **WINDOW = 240** (4 min).
- **STRIDE** configurável via `--step` (script `train_lstm.py`).
A célula acima já utilizou `window_size=240` e `step_size=10` para gerar janelas.

## 4. Estratégia para Número Variável de Sensores – Opção B
*LSTM compartilhada – um sensor por vez.*
Todos os sensores são empilhados num único dataset; o mesmo encoder LSTM processa qualquer sensor novo sem necessidade de re-treino.

## 5. Arquitetura 1-D (LSTM) e Sumário

In [ ]:
from codes.LSTM import LSTMModel
model_wrap = LSTMModel(window_size=240, metrics=[], debug=False)
model_wrap.model.summary()

## 6. Perda e Métricas
A escolha é `BinaryFocalCrossentropy` (flag `--focal`) ou `binary_crossentropy`.
As métricas estão definidas em `LSTM.F1Score`, `Precision`, `Recall`.

## 7. Callbacks de Treinamento
EarlyStopping (pacience ≥ 20) e ReduceLROnPlateau já configurados.

In [ ]:
callbacks = model_wrap.setup_callbacks('demo')
for cb in callbacks:
    print(type(cb).__name__)

**Conclusão** – Todos os requisitos obrigatórios do enunciado são atendidos pelo código atual.